In [52]:
# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [53]:
df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0, low_memory=False)

In [54]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')

In [55]:
df = df.iloc[:119390]

In [56]:
# →  Estandarizar para que en todos los casos sean números y cambiar el datatype de la columna a número entero.

def cambiar_formato_mes(df):
    # Diccionario de mapeo de meses y valores numéricos
    dic_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
        'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 
        'November': 11, 'December': 12, 
        1: 1, 2: 2, 3: 3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12, 'nan': np.nan}
    
    # Asegurarse de que los valores en la columna sean de tipo string para mapear correctamente
    df['arrival_date_month'] = df['arrival_date_month'].astype(str).map(dic_map).astype('Int64')

    return df

df = cambiar_formato_mes(df)

In [58]:


def rellenar_fecha_llegada(df):  

    print(f'Nulos antes de hacer la operación:')
    print(f'- arrival_date_year: {df["arrival_date_year"].isna().sum()}')
    print(f'- arrival_date_month: {df["arrival_date_month"].isna().sum()}')
    print(f'- arrival_date_day_of_month: {df["arrival_date_day_of_month"].isna().sum()}')

    # Asegúrate de que las columnas de fecha estén en formato datetime
    df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], errors='coerce')

    # Crear una columna para el total de la estancia
    df['total_stays'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

    # Inicializar la columna de fecha de llegada estimada con valores NaT (Not a Time)
    df['estimated_arrival_date'] = pd.NaT

    # Crear filtro para cuando el estado de la reserva es "Checkout"
    filtro_checkout = df['reservation_status'] == 'Checkout'

    # Calcular fecha de llegada estimada para "Checkout"
    df.loc[filtro_checkout, 'estimated_arrival_date'] = df.loc[filtro_checkout, 'reservation_status_date'] - pd.to_timedelta(df.loc[filtro_checkout, 'total_stays'], unit='D')

    # Para los que no son "Checkout", asumir que la llegada fue la fecha del estado de la reserva
    df.loc[~filtro_checkout, 'estimated_arrival_date'] = df.loc[~filtro_checkout, 'reservation_status_date']

    # Extraer el año, mes y día de la fecha de llegada estimada
    df['estimated_arrival_year'] = df['estimated_arrival_date'].dt.year
    df['estimated_arrival_month'] = df['estimated_arrival_date'].dt.month
    df['estimated_arrival_day'] = df['estimated_arrival_date'].dt.day

    # Rellenar los valores nulos en arrival_date_year, arrival_date_month y arrival_date_day_of_month con los valores estimados
    df['arrival_date_year'] = df['arrival_date_year'].fillna(df['estimated_arrival_year'])
    df['arrival_date_month'] = df['arrival_date_month'].fillna(df['estimated_arrival_month'])
    df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].fillna(df['estimated_arrival_day'])

    print(f'Nulos después de hacer la operación:')
    print(f'- arrival_date_year: {df["arrival_date_year"].isna().sum()}')
    print(f'- arrival_date_month: {df["arrival_date_month"].isna().sum()}')
    print(f'- arrival_date_day_of_month: {df["arrival_date_day_of_month"].isna().sum()}')

    # borramos las columnas creadas para hacer los cálculos
    columnas_a_borrar = ['estimated_arrival_date', 'estimated_arrival_year', 'estimated_arrival_month', 'estimated_arrival_day']
    df = df.drop(columns=columnas_a_borrar)

   


In [59]:
rellenar_fecha_llegada(df)

Nulos antes de hacer la operación:
- arrival_date_year: 54561
- arrival_date_month: 9390
- arrival_date_day_of_month: 119
Nulos después de hacer la operación:
- arrival_date_year: 5936
- arrival_date_month: 1042
- arrival_date_day_of_month: 13


In [60]:
def imputar_knn_fechas(df, n_neighbors=5):
    # Seleccionar las columnas relevantes para la imputación
    cols_fecha = ['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']
    
    # Filtrar el DataFrame para las columnas de fechas y convertirlas a float para KNNImputer
    df_fechas = df[cols_fecha].astype(float)
    
    # Imputar los valores nulos usando KNN
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputado = imputer.fit_transform(df_fechas)
    
    # Convertir el resultado de vuelta a un DataFrame con las mismas columnas
    df_imputado = pd.DataFrame(df_imputado, columns=cols_fecha)
    
    # Asegurarse de que los valores imputados sean enteros
    df_imputado = df_imputado.round().astype(int)
    
    # Reemplazar las columnas originales en el DataFrame con los valores imputados
    df[cols_fecha] = df_imputado
    
    

In [61]:
imputar_knn_fechas(df)

In [62]:
print(f'Nulos después de hacer la operación:')
print(f'- arrival_date_year: {df["arrival_date_year"].isna().sum()}')
print(f'- arrival_date_month: {df["arrival_date_month"].isna().sum()}')
print(f'- arrival_date_day_of_month: {df["arrival_date_day_of_month"].isna().sum()}')

Nulos después de hacer la operación:
- arrival_date_year: 0
- arrival_date_month: 0
- arrival_date_day_of_month: 0


In [69]:
#creamos una columa nueva con la fecha de llegada

def crear_columna_arrival_date(df):
  # Crear la columna de fecha combinada
      # Crear la columna de fecha combinada
    df['arrival_date'] = pd.to_datetime(
        df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(
            str
        ).agg('-'.join, axis=1), 
        format='%Y-%m-%d', 
        errors='coerce'
    )

In [70]:
crear_columna_arrival_date(df)